In [1]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [3]:
from sklearn.preprocessing import LabelEncoder
lbl = LabelEncoder()
df['time']=lbl.fit_transform(df['time'])
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [4]:
#Independent and Dependent features
x=df.drop(['time'],axis=1)
y=df['time']

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)

In [6]:
from sklearn.impute import SimpleImputer ## Handling Missing Values
from sklearn.preprocessing import OneHotEncoder ## handling Categorical features
from sklearn.preprocessing import StandardScaler ## Feature scaling
from sklearn.pipeline import Pipeline ## Automating the entire process
from sklearn.compose import ColumnTransformer

In [7]:
categorical_cols = ['sex','smoker','day']
numerical_cols = ['total_bill', 'tip', 'size']

# Automating Feature Engineering

In [8]:
# For numerical Features

num_pipeline = Pipeline(steps = [                    # For numerical columns
    ('imputer', SimpleImputer(strategy='median')),   # Handling Missing Values
    ('scaler', StandardScaler())                     # Feature Scaling
])

#For categorical features

cat_pipeline = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')), # Replacing missing values with mode of that column
    ('onehotencoder', OneHotEncoder())                    # Categorical features to numerical
])

In [9]:
preprocessor = ColumnTransformer([
    ('num_pipeline', num_pipeline, numerical_cols),
    ('cat_pipeline', cat_pipeline, categorical_cols)
])

In [10]:
x_train = preprocessor.fit_transform(x_train)
x_test = preprocessor.transform(x_test)

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [40]:
models = {
    'Random Forest' : RandomForestClassifier(),
    'Decision Tree' : DecisionTreeClassifier(),
    'SVC'           : SVC()
}

In [13]:
from sklearn.metrics import accuracy_score

In [35]:
def evaluate_model(x_train,y_train,x_test,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        
        # Train model
        model.fit(x_train,y_train)

            

        # Predict Testing data
        y_test_pred =model.predict(x_test)

        # Get accuracy for test data prediction
       
        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  str(test_model_score*100) + ' percent'
            

            
    return report

In [41]:
evaluate_model(x_train,y_train,x_test,y_test,models)

{'Random Forest': '95.91836734693877 percent',
 'Decision Tree': '93.87755102040816 percent',
 'SVC': '95.91836734693877 percent'}

In [42]:
classifier = RandomForestClassifier()

In [43]:
#Hyperprameter Tuning

params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [44]:
from sklearn.model_selection import RandomizedSearchCV

In [48]:
cv = RandomizedSearchCV(classifier, param_distributions=params, verbose=True, cv=5, scoring='accuracy')
cv.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=True)

In [49]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 10, 'criterion': 'gini'}